
# Superstore Sales & Profit Analysis (2015)

**Author:** _Neelam Kanwar_  
**Goal:** Analyze sales and profitability patterns for a retail superstore to uncover business insights and recommendations.

**Dataset:** `SuperStoreUS-2015.xlsx` (1,952 rows, 25 columns)



## 1) Setup & Data Loading
This section imports libraries and loads the dataset. If you're running this outside of this workspace, update `file_path` to your local path.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.max_columns', None)

# Load data
file_path = "SuperStoreUS-2015.xlsx"  # Update path as needed
df = pd.read_excel(file_path, engine="openpyxl")

# Quick peek
print(df.shape)
df.head()


In [ ]:

# Basic dtype inspection
df.dtypes



## 2) Cleaning
- Drop duplicates  
- Parse dates  
- Inspect missing values


In [ ]:

# Drop duplicates
df = df.drop_duplicates()

# Ensure dates
for col in ["Order Date", "Ship Date"]:
    if not np.issubdtype(df[col].dtype, np.datetime64):
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Missing values summary
missing_summary = df.isnull().sum().sort_values(ascending=False)
missing_summary[missing_summary > 0]



> Note: `Product Base Margin` has some missing values. For analysis below, we will leave them as-is (they are not required for our KPIs).



## 3) Core KPIs
Total Sales, Total Profit, Unique Customers, and Orders.


In [ ]:

kpis = {
    "Total Sales": df['Sales'].sum(),
    "Total Profit": df['Profit'].sum(),
    "Unique Customers": df['Customer ID'].nunique(),
    "Total Orders": df['Order ID'].nunique()
}
pd.DataFrame.from_dict(kpis, orient='index', columns=['Value'])



## 4) Exploratory Data Analysis (EDA)
This section explores sales and profit by category, region, time, products, and segments.


### 4.1 Sales by Category

In [ ]:

category_sales = (
    df.groupby('Product Category')['Sales']
    .sum()
    .sort_values(ascending=False)
)

plt.figure(figsize=(7,4))
plt.bar(category_sales.index, category_sales.values)
plt.title("Total Sales by Category")
plt.xlabel("Product Category")
plt.ylabel("Sales")
plt.tight_layout()
plt.show()

category_sales


### 4.2 Profit by Region

In [ ]:

region_profit = (
    df.groupby('Region')['Profit']
    .sum()
    .sort_values(ascending=False)
)

plt.figure(figsize=(7,4))
plt.bar(region_profit.index, region_profit.values)
plt.title("Total Profit by Region")
plt.xlabel("Region")
plt.ylabel("Profit")
plt.tight_layout()
plt.show()

region_profit


### 4.3 Monthly Sales Trend (2015)

In [ ]:

# Aggregate by month
df['Month'] = df['Order Date'].dt.to_period('M')
monthly_sales = df.groupby('Month')['Sales'].sum()

# Convert PeriodIndex to timestamp for plotting
x = monthly_sales.index.to_timestamp()
y = monthly_sales.values

plt.figure(figsize=(10,5))
plt.plot(x, y, marker='o')
plt.title("Monthly Sales Trend (2015)")
plt.xlabel("Month")
plt.ylabel("Sales")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

monthly_sales


### 4.4 Top 10 Products by Sales

In [ ]:

top_products_sales = (
    df.groupby('Product Name')['Sales']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(9,6))
plt.barh(top_products_sales.index[::-1], top_products_sales.values[::-1])
plt.title("Top 10 Products by Sales")
plt.xlabel("Sales")
plt.ylabel("Product Name")
plt.tight_layout()
plt.show()

top_products_sales


### 4.5 Top 10 Products by Profit

In [ ]:

top_products_profit = (
    df.groupby('Product Name')['Profit']
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

plt.figure(figsize=(9,6))
plt.barh(top_products_profit.index[::-1], top_products_profit.values[::-1])
plt.title("Top 10 Products by Profit")
plt.xlabel("Profit")
plt.ylabel("Product Name")
plt.tight_layout()
plt.show()

top_products_profit


### 4.6 Sales by Customer Segment

In [ ]:

segment_sales = (
    df.groupby('Customer Segment')['Sales']
    .sum()
    .sort_values(ascending=False)
)

plt.figure(figsize=(7,4))
plt.bar(segment_sales.index, segment_sales.values)
plt.title("Total Sales by Customer Segment")
plt.xlabel("Customer Segment")
plt.ylabel("Sales")
plt.tight_layout()
plt.show()

segment_sales


### 4.7 Discount vs Profit

In [ ]:

plt.figure(figsize=(7,4))
plt.scatter(df['Discount'], df['Profit'], alpha=0.6)
plt.title("Discount vs Profit")
plt.xlabel("Discount")
plt.ylabel("Profit")
plt.tight_layout()
plt.show()



## 5) Key Insights
- **Category Performance:** Technology and Furniture lead sales; cross-check their profit contribution above.
- **Regional Profitability:** Identify regions with negative or weak profit; investigate discounting and shipping costs.
- **Seasonality:** Spot months with spikes/dips; consider promotions or stock planning.
- **Product Mix:** Top products by sales are not always top by profit—optimize assortment and pricing.
- **Discount Impact:** Higher discounts tend to reduce profit; set guardrails.


In [ ]:

# Helpful one-liners to surface highlights programmatically
print("Top Category by Sales:", category_sales.index[0], f"(${category_sales.iloc[0]:,.2f})")
print("Top Region by Profit:", region_profit.index[0], f"(${region_profit.iloc[0]:,.2f})")
print("Best Month by Sales:", monthly_sales.idxmax().strftime('%b %Y'), f"(${monthly_sales.max():,.2f})")
print("Worst Month by Sales:", monthly_sales.idxmin().strftime('%b %Y'), f"(${monthly_sales.min():,.2f})")
print("\nMost Profitable Product:")
print(top_products_profit.head(1))
print("\nHighest Revenue Product:")
print(top_products_sales.head(1))



## 6) Recommendations
- **Tighten Discounting** in low-margin categories/regions.
- **Focus Marketing** on high-profit products and top-performing segments.
- **Investigate Loss-Making Regions** (e.g., high shipping cost, heavy discounting).
- **Inventory Planning** for peak months; pre-position stock to reduce shipping costs.
- **Customer Strategy:** nurture high-value customers with targeted offers that protect margin.
